In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

loss_func = 'categorical_crossentropy'

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from os import listdir
from keras.models import load_model

import numpy as np
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from os.path import isfile, join

import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, GlobalAveragePooling2D, BatchNormalization, Flatten, LSTM, TimeDistributed
from keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint

In [ ]:
# Define the teacher model
teacher_model = load_model('/content/drive/MyDrive/Colab Notebooks/teacher_model.h5')
# Define the student model
student_model = keras.Sequential()
img_shape = (255,255,3)

student_model.add(Conv2D(32, kernel_size=(5,5), strides=1, padding='same', input_shape= img_shape, activation='relu'))
student_model.add(BatchNormalization())
student_model.add(MaxPooling2D(pool_size=(2,2)))

student_model.add(Conv2D(64, kernel_size=5 ,strides=1, padding='same', activation='relu'))
student_model.add(BatchNormalization())
student_model.add(MaxPooling2D(pool_size=(2,2)))

student_model.add(Dropout(0.3))
student_model.add(Flatten())

student_model.add(Dense(64, activation='relu'))
student_model.add(Dense(32, activation='relu'))
student_model.add(Dense(6, activation='softmax'))
# Define the temperature for distillation
temperature = 7
#Learning Rate
alpha = 0.1

In [ ]:
path='/content/drive/MyDrive/db2'
directory = listdir(path)
le = LabelEncoder()
le.fit(directory)
x = []
y = []
for dic in directory:
  imgpath = path+'/'+dic
  imgs = listdir(imgpath)
  for i in imgs:
    src = imgpath+'/'+i
    img = tf.keras.utils.load_img(src,target_size=(255,255,3))
    

    img=np.array(img)
    img=img/255.0
    
    x.append(img)
    y.append(dic)


In [ ]:
def knowledge_distillation_loss(y_true, y_pred):
    y_true_soft, y_true_hard = tf.split(y_true, num_or_size_splits=2, axis=0)
    y_pred_soft, y_pred_hard = tf.split(y_pred, num_or_size_splits=2, axis=0)

    soft_targets_pred = keras.activations.softmax(y_pred_soft / temperature, axis=-1)
    soft_targets_true = keras.activations.softmax(y_true_soft / temperature,axis=-1)
    #distillation loss using softmax 
    distillation_loss = keras.losses.kullback_leibler_divergence(soft_targets_pred, soft_targets_true)
    #student loss using hardlabels
    student_loss = keras.losses.categorical_crossentropy(y_true_hard, y_pred_hard)

    total_loss = (1 - alpha) * student_loss + alpha * temperature**2 * distillation_loss
    return total_loss

In [ ]:
# Define the training data and labels
x_train = x[:916]
y_train = y[:916]

In [ ]:
# Split the training data into teacher and student inputs
x_train_soft, x_train_hard = tf.split(x_train, num_or_size_splits=2, axis=0)
y_train_soft, y_train_hard = tf.split(y_train, num_or_size_splits=2, axis=0)

In [ ]:
#Define the labels for knowledge distillation
y_train_soft = teacher_model.predict(x_train_soft)

y_train = tf.concat(tf.cast(tf.convert_to_tensor(y_train_soft)), y_train_hard,tf.float(),axis=1)

In [ ]:
student_model.compile(loss=loss_func, metrics=['accuracy'], optimizer='adam') 
student_model.fit(x, y, batch_size=128, epochs=10, validation_split=0.2)

Epoch 1/10
3/3 [==============================] - 124s 41s/step - loss: 2.3757 - accuracy: 0.6639 - val_loss: 0.0011 - val_accuracy: 1.0000
Epoch 2/10
3/3 [==============================] - 112s 37s/step - loss: 2.1309 - accuracy: 0.9044 - val_loss: 1.1434e-04 - val_accuracy: 1.0000
Epoch 3/10
3/3 [==============================] - 113s 38s/step - loss: 0.6468 - accuracy: 0.9945 - val_loss: 1.5559e-05 - val_accuracy: 1.0000
Epoch 4/10
3/3 [==============================] - 108s 37s/step - loss: 0.7247 - accuracy: 0.9945 - val_loss: 1.8595e-04 - val_accuracy: 1.0000
Epoch 5/10
3/3 [==============================] - 110s 37s/step - loss: 0.3790 - accuracy: 0.9945 - val_loss: 0.0049 - val_accuracy: 1.0000
Epoch 6/10
3/3 [==============================] - 108s 36s/step - loss: 0.2186 - accuracy: 0.9809 - val_loss: 0.0468 - val_accuracy: 0.9783
Epoch 7/10
3/3 [==============================] - 109s 38s/step - loss: 0.1569 - accuracy: 0.9945 - val_loss: 0.1190 - val_accuracy: 0.9565
Epoch 8/